# Импорт библиотек

In [1]:
import numpy as np       # линейная алгебра
import pandas as pd      # работа с табличными данными
import string            # работа со строковыми значениями
import re                # работа с регулярными выражениями

import nltk                               # работа с естественным языком           
from nltk.corpus import stopwords         # список стоп-слов
from nltk.stem.snowball import SnowballStemmer # стемизазия

from sklearn.feature_extraction.text import TfidfVectorizer          # Преобразование текстов в TFIDF-матрицу
from sklearn.metrics.pairwise import cosine_similarity               # Метрика измерения косинусного расстояния между векторами

import gensim.downloader                # работа с word2vec 


# Функции очистки текста

In [2]:
# Функция удаления символов из текста
def remove_chars_from_text(text_, chars_):
    return "".join([ch for ch in text_ if ch not in chars_])


# Функция удаления стоп-слов
def clear_stop_words(text_, stopwords_):
    return ' '.join([word for word in str(text_).split() if word not in stopwords_])


# Функция удаления HTML-ссылок
def remove_urls(text_):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r' ', text_)


# Функция удаления URL-ссылок
def remove_html(text_):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text_)

# Stemming
def stem_on(text_):
    stemer = SnowballStemmer('english')
    tokens = nltk.word_tokenize(text_)
    stemming = [stemer.stem(w) for w in tokens]
    return " ".join(stemming)


# Функция предобработки текста
def text_preproc(
    series_             # На вход получает данные в формате pandas.Series

):          
    
    # Удаляем URL-ссылки
    series_ = series_.apply(lambda x:remove_urls(x))

    # Удаляем HTML-ссылки
    series_ = series_.apply(lambda x: remove_html(x))

    # Переводим символы к нижнему регистру
    series_ = series_.apply(lambda x: x.lower())

    # Удаляем пунктуацию и спец-символы
    series_ = series_.apply(lambda x: remove_chars_from_text(x, string.punctuation))

    # Удаляем цифры
    series_ = series_.apply(lambda x: remove_chars_from_text(x, string.digits))

    # Убираем лишние пробелы
    series_ = series_.apply(lambda x: ' '.join(x.split()))
    
    # Убираем стоп-слова
    series_ = series_.apply(lambda x: clear_stop_words(x, stopwords.words('english')))
    
    # Stemming
    series_ = series_.apply(lambda x: stem_on(x))
    
    return series_


# Загрузка и обзор данных

In [3]:
# Загрузим данные
df = pd.read_csv('sample-data.csv')


In [4]:
# Пример данных
df.head(20)

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."
5,6,Ascensionist jkt - Our most technical soft she...
6,7,"Atom - A multitasker's cloud nine, the Atom pl..."
7,8,Print banded betina btm - Our fullest coverage...
8,9,Baby micro d-luxe cardigan - Micro D-Luxe is a...
9,10,Baby sun bucket hat - This hat goes on when th...


## Перед преобразованием в векторное представление тексты необходимо почистить.

In [5]:
# Проведем предобработку текста поля 'description'
df['description'] = text_preproc(df['description'])

# Проверим результат предобратобки
df['description'][0]

'activ classic boxer there reason boxer cult favorit keep cool especi sticki situat quickdri lightweight underwear take minim space travel pack expos brush waistband offer nexttoskin soft fivepanel construct tradit boxer back classic fit function fli made oz recycl polyest moisturewick perform inseam size recycl common thread recycl program detail silki capilen fabric ultralight breathabl quicktodri expos brush elast waistband comfort panel construct tradit boxer back inseam size fabric oz allrecycl polyest gladiodor natur odor control garment recycl common thread recycl program weight g oz made mexico'

# Векторизация текстов TF-IDF

In [6]:
# Инициализируем токенайзер для преобразования каждого текста в вектор. Максимальный размер словаря 25000 слов
tfidf_vectorizer = TfidfVectorizer(binary=True, max_features=25000)

# Преобразуем очищенные тексты в матрицы TFIDF
tfidf_embedings = tfidf_vectorizer.fit_transform(df['description'])

# Проверяем размерность полученного массива
tfidf_embedings.shape

(500, 3916)

# Считаем метрику косинусного расстояния с помощью функции cosine_similarity() библиотеки scikit-learn

In [7]:
# Вычисляем попарно косинусное расстояние между векторами в матрице, выделяем нижнюю треугольную матрицу, чтобы пара векторов встречалась 1 раз.
cos_matrix = np.tril(cosine_similarity(tfidf_embedings))

In [8]:
cos_matrix

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.1406852 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10830019, 0.47592662, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03573089, 0.02240297, 0.0292735 , ..., 1.        , 0.        ,
        0.        ],
       [0.07620833, 0.11161524, 0.04335247, ..., 0.03850091, 1.        ,
        0.        ],
       [0.0701705 , 0.0410301 , 0.03078927, ..., 0.03769678, 0.33301946,
        1.        ]])

In [9]:
pair_indexes_tf_idf = np.transpose(np.nonzero(cos_matrix > 0.8))
description_pairs_tf_idf = pd.DataFrame()

i = 0

# Перебираем пары индексов
for x in pair_indexes_tf_idf:

    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:

        # Выводим пары описаний товаров
        pairs = pd.DataFrame({'index_pairs':[(x[0], x[1])], 'description_pairs':[(df['description'][x[0]], df['description'][x[1]])], 'cosine_similarity_score':[cos_matrix[x[0], x[1]]]})
        description_pairs_tf_idf = pd.concat([description_pairs_tf_idf, pairs], ignore_index=True)
        i += 1
print(f'Количество пар похожих товаров - {i}')
description_pairs_tf_idf

Количество пар похожих товаров - 84


,index_pairs,description_pairs,cosine_similarity_score
0,"(27, 26)",(compound cargo pant reg ultim doeveryth pant ...,0.990242
1,"(62, 57)",(fli fish tshirt softwear ringspun organ cotto...,0.929723
2,"(63, 57)",(gpiw classic tshirt softwear ringspun organ c...,0.882237
3,"(63, 62)",(gpiw classic tshirt softwear ringspun organ c...,0.862975
4,"(64, 57)",(live simpli guitar tshirt softwear ringspun o...,0.865243
...,...,...,...
79,"(480, 36)",(duck pant reg essenti wear split log drive na...,0.985911
80,"(481, 36)",(duck pant short essenti wear split log drive ...,1.000000
81,"(481, 480)",(duck pant short essenti wear split log drive ...,0.985911
82,"(485, 455)",(half mass size ride bike fine tune hold day w...,0.811473


# Считаем метрику косинусного расстояния вручную путем матричного умножения

In [10]:
tfidf_m = tfidf_embedings.toarray()
def cosine_similarity_m(X):
    norm_X = np.linalg.norm(X)
    norm_X_T = np.linalg.norm(X.T)
    similarity = np.dot(X, X.T)
    return similarity

In [11]:
cos_matrix_m = np.tril(cosine_similarity_m(tfidf_m))

In [12]:
cos_matrix_m

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.1406852 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10830019, 0.47592662, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03573089, 0.02240297, 0.0292735 , ..., 1.        , 0.        ,
        0.        ],
       [0.07620833, 0.11161524, 0.04335247, ..., 0.03850091, 1.        ,
        0.        ],
       [0.0701705 , 0.0410301 , 0.03078927, ..., 0.03769678, 0.33301946,
        1.        ]])

In [13]:
# Находим индексы значений косинусного расстояния больше заданного
pair_indexes_tf_idf = np.transpose(np.nonzero(cos_matrix_m > 0.8))
description_pairs_tf_idf = pd.DataFrame()
 
i = 0

# Перебираем пары индексов
for x in pair_indexes_tf_idf:
       
    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:
        
        # Выводим пары описаний товаров
        pairs = pd.DataFrame({'index_pairs':[(x[0], x[1])], 'description_pairs':[(df['description'][x[0]], df['description'][x[1]])], 'cosine_similarity_score':[cos_matrix[x[0], x[1]]]})
        description_pairs_tf_idf = pd.concat([description_pairs_tf_idf, pairs], ignore_index=True)
        i += 1
print(f'Количество пар похожих товаров - {i}')
description_pairs_tf_idf

Количество пар похожих товаров - 84


,index_pairs,description_pairs,cosine_similarity_score
0,"(27, 26)",(compound cargo pant reg ultim doeveryth pant ...,0.990242
1,"(62, 57)",(fli fish tshirt softwear ringspun organ cotto...,0.929723
2,"(63, 57)",(gpiw classic tshirt softwear ringspun organ c...,0.882237
3,"(63, 62)",(gpiw classic tshirt softwear ringspun organ c...,0.862975
4,"(64, 57)",(live simpli guitar tshirt softwear ringspun o...,0.865243
...,...,...,...
79,"(480, 36)",(duck pant reg essenti wear split log drive na...,0.985911
80,"(481, 36)",(duck pant short essenti wear split log drive ...,1.000000
81,"(481, 480)",(duck pant short essenti wear split log drive ...,0.985911
82,"(485, 455)",(half mass size ride bike fine tune hold day w...,0.811473


In [14]:
description_pairs_tf_idf.to_csv('similar_products_tf_idf.csv', index=False)

 # Векторизация текстов W2V

In [15]:
# Загружаем предобученную модель
word_vectors = gensim.downloader.load("word2vec-google-news-300")

In [16]:
# Создаем пустой DataFrame для эмбедингов текстов
w2v_embeddings = pd.DataFrame()

# Идем по текстам
for doc in df['description']:
    
    # Создаем пустой DataFrame для хранения векторов слов текста
    temp = pd.DataFrame()
    
    # Идем по словам в тексте
    for word in doc.split(' '):
        
        try:
            # Векторизуем слово с использованием предобученной модели W2V
            word_vec = word_vectors[word]

            word_vec_df = pd.DataFrame(word_vec).T

            # Добавляем вектор слова в DF
            temp = pd.concat([temp, word_vec_df], ignore_index = True)
            
        except:
            pass
        
    # Вычисляем вектор текста усредняя массив векторов слов
    doc_vector = temp.mean(axis=0).to_frame().T
    
    # Добавляем вектор текста в DF эмбедингов текстов
    w2v_embeddings = pd.concat([w2v_embeddings, doc_vector], ignore_index = True)

# Размерность DF эмбедингов текстов    
w2v_embeddings.shape

(500, 300)

In [17]:
# Вычисляем попарно косинусное расстояние между векторами в матрице, округляем значения до 5 знака после запятой
cos_matrix_w2v = np.tril(cosine_similarity(w2v_embeddings))
cos_matrix_w2v

array([[1.0000002 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.8957728 , 1.0000002 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.8277148 , 0.92433214, 0.9999999 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.80830157, 0.8535694 , 0.82517743, ..., 0.9999999 , 0.        ,
        0.        ],
       [0.834347  , 0.86512613, 0.80959666, ..., 0.85467243, 1.        ,
        0.        ],
       [0.84218127, 0.86168325, 0.8122253 , ..., 0.80746603, 0.94414234,
        0.9999995 ]], dtype=float32)

In [18]:
# Находим пары векторов, косинусное расстояние между которыми больше 0.985
pair_indexes_w2v = np.transpose(np.nonzero(cos_matrix_w2v > 0.985))
i = 0
# Перебираем пары индексов
for x in pair_indexes_w2v:
    
    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:
        i += 1
        # Выводим пары похожих описаний товаров
        print(df['description'][x])
print(f'Количество пар похожих товаров - {i}')

15    borderless short one summertim gift chanc free...
14    borderless short go forward other cring border...
Name: description, dtype: object
27    compound cargo pant reg ultim doeveryth pant b...
26    compound cargo pant long ultim doeveryth pant ...
Name: description, dtype: object
62    fli fish tshirt softwear ringspun organ cotton...
57    logo tshirt softwear ringspun organ cotton scr...
Name: description, dtype: object
63    gpiw classic tshirt softwear ringspun organ co...
57    logo tshirt softwear ringspun organ cotton scr...
Name: description, dtype: object
63    gpiw classic tshirt softwear ringspun organ co...
62    fli fish tshirt softwear ringspun organ cotton...
Name: description, dtype: object
64    live simpli guitar tshirt softwear ringspun or...
57    logo tshirt softwear ringspun organ cotton scr...
Name: description, dtype: object
64    live simpli guitar tshirt softwear ringspun or...
62    fli fish tshirt softwear ringspun organ cotton...
Name: description,

In [19]:
pair_indexes_w2v = np.transpose(np.nonzero(cos_matrix_w2v > 0.99))
description_pairs_w2v = pd.DataFrame()

i = 0

# Перебираем пары индексов
for x in pair_indexes_w2v:

    # Если индексы не равны друг другу, что означает, что сравнивались описания разных товаров
    if x[0] != x[1]:

        # Выводим пары описаний товаров
        pairs = pd.DataFrame({'index_pairs':[(x[0], x[1])], 'description_pairs':[(df['description'][x[0]], df['description'][x[1]])], 'cosine_similarity_score':[cos_matrix[x[0], x[1]]]})
        description_pairs_w2v = pd.concat([description_pairs_w2v, pairs], ignore_index=True)
        i += 1
print(f'Количество пар похожих товаров - {i}')
description_pairs_w2v

Количество пар похожих товаров - 83


,index_pairs,description_pairs,cosine_similarity_score
0,"(27, 26)",(compound cargo pant reg ultim doeveryth pant ...,0.990242
1,"(62, 57)",(fli fish tshirt softwear ringspun organ cotto...,0.929723
2,"(63, 57)",(gpiw classic tshirt softwear ringspun organ c...,0.882237
3,"(63, 62)",(gpiw classic tshirt softwear ringspun organ c...,0.862975
4,"(64, 62)",(live simpli guitar tshirt softwear ringspun o...,0.846352
...,...,...,...
78,"(480, 36)",(duck pant reg essenti wear split log drive na...,0.985911
79,"(481, 36)",(duck pant short essenti wear split log drive ...,1.000000
80,"(481, 480)",(duck pant short essenti wear split log drive ...,0.985911
81,"(485, 455)",(half mass size ride bike fine tune hold day w...,0.811473


In [20]:
description_pairs_w2v.to_csv('similar_products_w2v.csv', index=False)

# Метод TF-IDF преобразует тексты в векторы более разреженные, а метод Word2Vec - в более плотные. Расстояние между векторами, полученными методом TF-IDF, отражает словарное сходство текстов, тогда как расстояние между векторами, полученными методом Word2Vec, отражает семантическое сходство текстов. 
# Мы видим, чтобы получить примерно одинаковые пары схожих товаров, порог значимости, используемый с Word2Vec, берется намного выше, чем используемый с TF-IDF.